In [1]:
"""
Purpose: To package up the Soma Finder into a Robust Module that can be used on the cluster 
(No file location errors)

"""

'\nPurpose: To package up the Soma Finder into a Robust Module that can be used on the cluster \n(No file location errors)\n\n'

In [2]:
import cgal_Segmentation_Module as csm
from whole_neuron_classifier_datajoint_adapted import extract_branches_whole_neuron
import whole_neuron_classifier_datajoint_adapted as wcda 
import time
import trimesh
import numpy as np
import datajoint as dj
import os
from meshlab import Decimator , Poisson
from pathlib import Path
from pykdtree.kdtree import KDTree

In [3]:
# ex_mesh = total_soma_list_revised_sphere[0]
# #[p for p in dir(ex_mesh). if "box" in p] #['bounding_box', 'bounding_box_oriented']
# ex_mesh.bounding_box.volume>ex_mesh.bounding_box_oriented.volume
# ex_mesh.bounding_box_oriented.is_oriented

In [4]:
"""
Checking the new validation checks
"""
def side_length_ratios(current_mesh):
    """
    Will compute the ratios of the bounding box sides
    To be later used to see if there is skewness
    """

    # bbox = current_mesh.bounding_box_oriented.vertices
    bbox = current_mesh.bounding_box_oriented.vertices
    x_axis_unique = np.unique(bbox[:,0])
    y_axis_unique = np.unique(bbox[:,1])
    z_axis_unique = np.unique(bbox[:,2])
    x_length = (np.max(x_axis_unique) - np.min(x_axis_unique)).astype("float")
    y_length = (np.max(y_axis_unique) - np.min(y_axis_unique)).astype("float")
    z_length = (np.max(z_axis_unique) - np.min(z_axis_unique)).astype("float")
    #print(x_length,y_length,z_length)
    #compute the ratios:
    xy_ratio = float(x_length/y_length)
    xz_ratio = float(x_length/z_length)
    yz_ratio = float(y_length/z_length)
    side_ratios = [xy_ratio,xz_ratio,yz_ratio]
    flipped_side_ratios = []
    for z in side_ratios:
        if z < 1:
            flipped_side_ratios.append(1/z)
        else:
            flipped_side_ratios.append(z)
    return flipped_side_ratios

def side_length_check(current_mesh,side_length_ratio_threshold=3):
    side_length_ratio_names = ["xy","xz","yz"]
    side_ratios = side_length_ratios(current_mesh)
    pass_threshold = [(k <= side_length_ratio_threshold) and
                      (k >= 1/side_length_ratio_threshold) for k in side_ratios]
    for i,(rt,truth) in enumerate(zip(side_ratios,pass_threshold)):
        if not truth:
            print(f"{side_length_ratio_names[i]} = {rt} ratio was beyong {side_length_ratio_threshold} multiplier")

    if False in pass_threshold:
        return False
    else:
        return True
# current_Pmesh = trimesh.load_mesh("./12346/12346_poisson_soma.off")
# print(side_length_ratios(current_mesh))
# side_length_check(current_mesh,side_length_ratio_threshold=1.55)



In [5]:
def largest_mesh_piece(msh):
    mesh_splits_inner = msh.split(only_watertight=False)
    total_mesh_split_lengths_inner = [len(k.faces) for k in mesh_splits_inner]
    ordered_mesh_splits_inner = mesh_splits_inner[np.flip(np.argsort(total_mesh_split_lengths_inner))]
    return ordered_mesh_splits_inner[0]
def soma_volume_ratio(current_mesh):
    """
    bounding_box_oriented: rotates the box to be less volume
    bounding_box : does not rotate the box and makes it axis aligned
    
    ** checks to see if closed mesh and if not then make closed **
    """
    poisson_temp_folder = Path.cwd() / "Poisson_temp"
    poisson_temp_folder.mkdir(parents=True,exist_ok=True)
    Poisson_obj_temp = Poisson(poisson_temp_folder,overwrite=True)
    
    #get the largest piece
    lrg_mesh = largest_mesh_piece(current_mesh)
    if not lrg_mesh.is_watertight:
        print("Using Poisson Surface Reconstruction to make mesh watertight")
        #run the Poisson Surface reconstruction and get the largest piece
        new_mesh_inner,poisson_file_obj = Poisson_obj_temp(vertices=lrg_mesh.vertices,
               faces=lrg_mesh.faces,
               return_mesh=True,
               delete_temp_files=True)
        lrg_mesh = largest_mesh_piece(new_mesh_inner)

    #turn the mesh into a closed mesh based on 
    
    ratio_val = lrg_mesh.bounding_box.volume/lrg_mesh.volume
#     if ratio_val < 1:
#         raise Exception("Less than 1 value in volume ratio computation")
    return ratio_val

def soma_volume_check(current_mesh,multiplier=8):
    ratio_val= soma_volume_ratio(current_mesh)
    print("Inside sphere validater: ratio_val = " + str(ratio_val))
    if np.abs(ratio_val) > multiplier:
        return False
    return True

# The actual soma extraction function

In [6]:
import trimesh

my_segment = 107738877133006848
#my_segment = 12347 #The dendrite branch
#my_segment = 12346 #the small glia
bc_mesh = trimesh.load_mesh(f"{my_segment}/{my_segment}.off")

segment_id = my_segment
current_mesh_verts = bc_mesh.vertices
current_mesh_faces = bc_mesh.faces

In [7]:
import time
global_start_time = time.time()

"""

"""

outer_decimation_ratio= 0.25
#large_mesh_threshold = 70000 ** this was working before but dropping down
large_mesh_threshold = 40000
large_mesh_threshold_inner = 40000
soma_width_threshold = 0.32
soma_size_threshold = 20000

large_mesh_threshold = large_mesh_threshold*outer_decimation_ratio
large_mesh_threshold_inner = large_mesh_threshold_inner*outer_decimation_ratio
soma_size_threshold = soma_size_threshold*outer_decimation_ratio

inner_decimation_ratio = 0.25
soma_size_threshold = soma_size_threshold*inner_decimation_ratio

print(f"Current Arguments Using (adjusted for decimation):\n large_mesh_threshold= {large_mesh_threshold}"
             f" \nlarge_mesh_threshold_inner = {large_mesh_threshold_inner} \nsoma_size_threshold = {soma_size_threshold}"
             f"\nouter_decimation_ratio = {outer_decimation_ratio}"
             f"\ninner_decimation_ratio = {inner_decimation_ratio}")


# ------------------------------


temp_folder = f"./{segment_id}"
temp_object = Path(temp_folder)
#make the temp folder if it doesn't exist
temp_object.mkdir(parents=True,exist_ok=True)

#making the decimation and poisson objections
Dec_outer = Decimator(outer_decimation_ratio,temp_folder,overwrite=True)
Dec_inner = Decimator(inner_decimation_ratio,temp_folder,overwrite=True)
Poisson_obj = Poisson(temp_folder,overwrite=True)

#Step 1: Decimate the Mesh and then split into the seperate pieces
new_mesh,output_obj = Dec_outer(vertices=current_mesh_verts,
         faces=current_mesh_faces,
         segment_id=segment_id,
         return_mesh=True,
         delete_temp_files=False)

#preforming the splits of the decimated mesh

mesh_splits = new_mesh.split(only_watertight=False)

#get the largest mesh
mesh_lengths = np.array([len(split.faces) for split in mesh_splits])


total_mesh_split_lengths = [len(k.faces) for k in mesh_splits]
ordered_mesh_splits = mesh_splits[np.flip(np.argsort(total_mesh_split_lengths))]
list_of_largest_mesh = [k for k in ordered_mesh_splits if len(k.faces) > large_mesh_threshold]

print(f"Total found significant pieces before Poisson = {list_of_largest_mesh}")

#if no significant pieces were found then will use smaller threshold
if len(list_of_largest_mesh)<=0:
    print(f"Using smaller large_mesh_threshold because no significant pieces found with {large_mesh_threshold}")
    list_of_largest_mesh = [k for k in ordered_mesh_splits if len(k.faces) > large_mesh_threshold/2]

total_soma_list = []
total_classifier_list = []
total_poisson_list = []

#start iterating through where go through all pieces before the poisson reconstruction
no_somas_found_in_big_loop = 0
for i,largest_mesh in enumerate(list_of_largest_mesh):
    print(f"----- working on large mesh #{i}: {largest_mesh}")

    somas_found_in_big_loop = False
    
    largest_file_name = str(output_obj.stem) + "_largest_piece.off"
    pre_largest_mesh_path = temp_object / Path(str(output_obj.stem) + "_largest_piece.off")
    pre_largest_mesh_path = pre_largest_mesh_path.absolute()
    print(f"pre_largest_mesh_path = {pre_largest_mesh_path}")
    
    new_mesh_inner,poisson_file_obj = Poisson_obj(vertices=largest_mesh.vertices,
               faces=largest_mesh.faces,
               return_mesh=True,
               mesh_filename=largest_file_name,
               delete_temp_files=False)
    
    
    #splitting the Poisson into the largest pieces and ordering them
    mesh_splits_inner = new_mesh_inner.split(only_watertight=False)
    total_mesh_split_lengths_inner = [len(k.faces) for k in mesh_splits_inner]
    ordered_mesh_splits_inner = mesh_splits_inner[np.flip(np.argsort(total_mesh_split_lengths_inner))]

    list_of_largest_mesh_inner = [k for k in ordered_mesh_splits_inner if len(k.faces) > large_mesh_threshold_inner]
    print(f"Total found significant pieces AFTER Poisson = {list_of_largest_mesh_inner}")

    n_failed_inner_soma_loops = 0
    for j, largest_mesh_inner in enumerate(list_of_largest_mesh_inner):
        print(f"----- working on mesh after poisson #{j}: {largest_mesh_inner}")
    
        largest_mesh_path_inner = str(poisson_file_obj.stem) + "_largest_inner.off"
        
        #Decimate the inner poisson piece
        largest_mesh_path_inner_decimated,output_obj_inner = Dec_inner(
                            vertices=largest_mesh_inner.vertices,
                             faces=largest_mesh_inner.faces,
                            mesh_filename=largest_mesh_path_inner,
                             return_mesh=True,
                             delete_temp_files=False)
        
        print(f"done exporting decimated mesh: {largest_mesh_path_inner}")
        
        faces = np.array(largest_mesh_path_inner_decimated.faces)
        verts = np.array(largest_mesh_path_inner_decimated.vertices)
        
        segment_id_new = int(str(segment_id) + f"{i}{j}")
        
        verts_labels, faces_labels, soma_value,classifier = wcda.extract_branches_whole_neuron(
                                import_Off_Flag=False,
                                segment_id=segment_id_new,
                                vertices=verts,
                                 triangles=faces,
                                pymeshfix_Flag=False,
                                 import_CGAL_Flag=False,
                                 return_Only_Labels=True,
                                 clusters=3,
                                 smoothness=0.2,
                                soma_only=True,
                                return_classifier = True
                                )
        print(f"soma_sdf_value = {soma_value}")
        
        total_classifier_list.append(classifier)
        #total_poisson_list.append(largest_mesh_path_inner_decimated)

        # Save all of the portions that resemble a soma
        median_values = np.array([v["median"] for k,v in classifier.sdf_final_dict.items()])
        segmentation = np.array([k for k,v in classifier.sdf_final_dict.items()])

        #order the compartments by greatest to smallest
        sorted_medians = np.flip(np.argsort(median_values))
        print(f"segmentation[sorted_medians],median_values[sorted_medians] = {(segmentation[sorted_medians],median_values[sorted_medians])}")
        print(f"Sizes = {[classifier.sdf_final_dict[g]['n_faces'] for g in segmentation[sorted_medians]]}")

        valid_soma_segments_width = [g for g,h in zip(segmentation[sorted_medians],median_values[sorted_medians]) if ((h > soma_width_threshold)
                                                            and (classifier.sdf_final_dict[g]["n_faces"] > soma_size_threshold))]

        print("valid_soma_segments_width")
        to_add_list = []
        if len(valid_soma_segments_width) > 0:
            print(f"      ------ Found {len(valid_soma_segments_width)} viable somas: {valid_soma_segments_width}")
            somas_found_in_big_loop = True
            #get the meshes only if signfiicant length
            labels_list = classifier.labels_list

            for v in valid_soma_segments_width:
                submesh_face_list = np.where(classifier.labels_list == v)[0]
                soma_mesh = largest_mesh_path_inner_decimated.submesh([submesh_face_list],append=True)

                if side_length_check(soma_mesh) and soma_volume_check(soma_mesh):
                    to_add_list.append(soma_mesh)
                else:
                    print(f"--->This soma mesh was not added because it did not pass the sphere validation: {soma_mesh}")

            n_failed_inner_soma_loops = 0

        else:
            n_failed_inner_soma_loops += 1

        total_soma_list += to_add_list

        # --------------- KEEP TRACK IF FAILED TO FIND SOMA (IF TOO MANY FAILS THEN BREAK)
        if n_failed_inner_soma_loops >= 2:
            print("breaking inner loop because 2 soma fails in a row")
            break


    # --------------- KEEP TRACK IF FAILED TO FIND SOMA (IF TOO MANY FAILS THEN BREAK)
    if somas_found_in_big_loop == False:
        no_somas_found_in_big_loop += 1
        if no_somas_found_in_big_loop >= 2:
            print("breaking because 2 fails in a row in big loop")
            break

    else:
        no_somas_found_in_big_loop = 0



    """
    large_mesh_threshold= 150000.0 
    large_mesh_threshold_inner = 10000.0 
    soma_size_threshold = 1250.0
    decimation_ratio = 0.05
    """

""" IF THERE ARE MULTIPLE SOMAS THAT ARE WITHIN A CERTAIN DISTANCE OF EACH OTHER THEN JUST COMBINE THEM INTO ONE"""
pairings = []
for y,soma_1 in enumerate(total_soma_list):
    for z,soma_2 in enumerate(total_soma_list):
        if y<z:
            mesh_tree = KDTree(soma_1.vertices)
            distances,closest_node = mesh_tree.query(soma_2.vertices)

            if np.min(distances) < 4000:
                pairings.append([y,z])


#creating the combined meshes from the list
total_soma_list_revised = []
if len(pairings) > 0:
    """
    Pseudocode: 
    Use a network function to find components

    """


    import networkx as nx
    new_graph = nx.Graph()
    new_graph.add_edges_from(pairings)
    grouped_somas = list(nx.connected_components(new_graph))

    somas_being_combined = []
    print(f"There were soma pariings: Connected components in = {grouped_somas} ")
    for comp in grouped_somas:
        comp = list(comp)
        somas_being_combined += list(comp)
        current_mesh = total_soma_list[comp[0]]
        for i in range(1,len(comp)):
            current_mesh += total_soma_list[comp[i]]

        total_soma_list_revised.append(current_mesh)

    #add those that weren't combined to total_soma_list_revised
    leftover_somas = [total_soma_list[k] for k in range(0,len(total_soma_list)) if k not in somas_being_combined]
    if len(leftover_somas) > 0:
        total_soma_list_revised += leftover_somas
    print(f"Final total_soma_list_revised = {total_soma_list_revised}")

    total_soma_list_revised

if len(total_soma_list_revised) == 0:
    total_soma_list_revised = total_soma_list

# # run through the sphere validator again (DON'T THINK WE NEED THIS ANYMORE)
# print(f"BEFORE last step sphere validation number of somas = {len(total_soma_list_revised)}")
# total_soma_list_revised_sphere = [m for m in total_soma_list_revised if validate_soma_by_sphere(m)]
# print(f"AFTER last step sphere validation number of somas = {len(total_soma_list_revised_sphere)}")

    
run_time = time.time() - global_start_time

print(f"\n\n\n Total time for run = {time.time() - global_start_time}")

#need to erase all of the temporary files ******
#import shutil
#shutil.rmtree(directory)

"""
Need to delete all files in the temp folder *****
"""


#return total_soma_list, run_time
        
        


Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 10000.0 
large_mesh_threshold_inner = 10000.0 
soma_size_threshold = 1250.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Soma/107738877133006848/neuron_107738877133006848.off -o /notebooks/Platinum_Soma/107738877133006848/neuron_107738877133006848_decimated.off -s /notebooks/Platinum_Soma/decimation_meshlab_25.mls
Total found significant pieces before Poisson = [<trimesh.Trimesh(vertices.shape=(327432, 3), faces.shape=(682632, 3))>]
----- working on large mesh #0: <trimesh.Trimesh(vertices.shape=(327432, 3), faces.shape=(682632, 3))>
pre_largest_mesh_path = /notebooks/Platinum_Soma/107738877133006848/neuron_107738877133006848_decimated_largest_piece.off
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver 

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_norma

Total found significant pieces AFTER Poisson = [<trimesh.Trimesh(vertices.shape=(284813, 3), faces.shape=(570782, 3))>, <trimesh.Trimesh(vertices.shape=(121397, 3), faces.shape=(243354, 3))>, <trimesh.Trimesh(vertices.shape=(44644, 3), faces.shape=(89284, 3))>, <trimesh.Trimesh(vertices.shape=(44175, 3), faces.shape=(88350, 3))>, <trimesh.Trimesh(vertices.shape=(5135, 3), faces.shape=(10278, 3))>]
----- working on mesh after poisson #0: <trimesh.Trimesh(vertices.shape=(284813, 3), faces.shape=(570782, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Soma/107738877133006848/neuron_107738877133006848_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Soma/107738877133006848/neuron_107738877133006848_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/Platinum_Soma/decimation_meshlab_25.mls
done exporting decimated mesh: neuron_107738877133006848_decima

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!


removed temporary input file: /notebooks/Platinum_Soma/Poisson_temp/neuron_None.off
removed temporary output file: /notebooks/Platinum_Soma/Poisson_temp/neuron_None_poisson.off
Inside sphere validater: ratio_val = 67.73408606998751
--->This soma mesh was not added because it did not pass the sphere validation: <trimesh.Trimesh(vertices.shape=(10261, 3), faces.shape=(20140, 3))>
----- working on mesh after poisson #1: <trimesh.Trimesh(vertices.shape=(121397, 3), faces.shape=(243354, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Soma/107738877133006848/neuron_107738877133006848_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Soma/107738877133006848/neuron_107738877133006848_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/Platinum_Soma/decimation_meshlab_25.mls
done exporting decimated mesh: neuron_107738877133006848_decimated_largest_piece_po

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!


removed temporary input file: /notebooks/Platinum_Soma/Poisson_temp/neuron_None.off
removed temporary output file: /notebooks/Platinum_Soma/Poisson_temp/neuron_None_poisson.off
Inside sphere validater: ratio_val = 2.878097383240724
----- working on mesh after poisson #2: <trimesh.Trimesh(vertices.shape=(44644, 3), faces.shape=(89284, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Soma/107738877133006848/neuron_107738877133006848_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Soma/107738877133006848/neuron_107738877133006848_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/Platinum_Soma/decimation_meshlab_25.mls
done exporting decimated mesh: neuron_107738877133006848_decimated_largest_piece_poisson_largest_inner.off
1) Starting: Mesh importing and Pymesh fix
loading mesh from vertices and triangles array
1) Finished: Mesh importing and Pymes

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!


removed temporary input file: /notebooks/Platinum_Soma/Poisson_temp/neuron_None.off
removed temporary output file: /notebooks/Platinum_Soma/Poisson_temp/neuron_None_poisson.off
Inside sphere validater: ratio_val = 3.196828682227806
----- working on mesh after poisson #4: <trimesh.Trimesh(vertices.shape=(5135, 3), faces.shape=(10278, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Soma/107738877133006848/neuron_107738877133006848_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Soma/107738877133006848/neuron_107738877133006848_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/Platinum_Soma/decimation_meshlab_25.mls
done exporting decimated mesh: neuron_107738877133006848_decimated_largest_piece_poisson_largest_inner.off
1) Starting: Mesh importing and Pymesh fix
loading mesh from vertices and triangles array
1) Finished: Mesh importing and Pymesh

'\nNeed to delete all files in the temp folder *****\n'

In [8]:
final_soma_mesh = trimesh.Trimesh(vertices=np.array([]),
                                 faces=np.array([]))
for sm in total_soma_list_revised:
    final_soma_mesh += sm
final_soma_mesh.export(f"{temp_folder}/{segment_id}_final_soma_meshes.off")
print("hi")

hi


In [9]:
total_soma_list_revised

[<trimesh.Trimesh(vertices.shape=(2281, 3), faces.shape=(4383, 3))>,
 <trimesh.Trimesh(vertices.shape=(1568, 3), faces.shape=(3044, 3))>,
 <trimesh.Trimesh(vertices.shape=(1936, 3), faces.shape=(3833, 3))>]